In [50]:
#from setup import *
#from agents import *
#from env import *
gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps = '1001', 2, 6, 3, 1, 10000, 9
disclosure = ['rnd', 'period', 'step','currentBid','currentAsk','buy','sell','price','sale']
buyerStrategies = ['Reinforcer', 'TruthTeller']
sellerStrategies = ['TruthTeller', 'TruthTeller','TruthTeller', 'TruthTeller','TruthTeller', 'TruthTeller']
metaData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, disclosure, buyerStrategies, sellerStrategies]
gameData = metaData[0:7]
gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps = gameData
disclosure, buyerStrategies, sellerStrategies = metaData[7:]
buyers, sellers = generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure)
log = Log(gameData, buyerStrategies, sellerStrategies, disclosure)

for rnd in range(numRounds):
    roundData = roundSetup(*gameData)
    redemptionValues, tokenCosts, demand, supply, prices, peq, qeq  = roundData[0:7]
    [buyerReservationPrices, sellerReservationPrices, buyerSurplus, sellerSurplus, totalSurplus, buyerSurplusFrac, sellerSurplusFrac] = roundData[7:]
    log.addRound([rnd] + roundData)
    resetRounds(buyers, sellers, redemptionValues, tokenCosts)
    for period in range(numPeriods):
        resetPeriods(buyers, sellers)
        for step in range(numSteps):
            resetSteps(buyers, sellers)
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose()) # observe new state
            updateStates(buyers, sellers)
        updatePolicy(buyers, sellers)
        
        if period % (numPeriods/20) == 0:
            print(f'\n{period}')
            display(log.getPeriod(rnd, period).tail(100)[['bprofit', 'currentBidIdx']].groupby('currentBidIdx').sum())
            display(log.getPeriod(rnd, period).tail(100)[['sprofit', 'currentAskIdx']].groupby('currentAskIdx').sum())
            display(log.getPeriod(rnd,period))


0


,bprofit
currentBidIdx,
0.0,122.85
1.0,109.40


,sprofit
currentAskIdx,
0,26.95
1,32.45
2,29.40
3,50.00
4,0.00
5,44.55


,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
0,0,0,0,"[52.0, 100.0]","[7.7, 2.7, 8.1, 0.0, 15.6, 1.9]",100.0,1.0,0.0,3,1.0,1.0,50.00,True,50.00,50.00
1,0,0,1,"[91.0, 67.6]","[7.7, 2.7, 8.1, 18.0, 15.6, 1.9]",91.0,0.0,1.9,5,1.0,1.0,46.45,True,48.65,44.55
2,0,0,2,"[32.0, 67.6]","[7.7, 2.7, 8.1, 18.0, 15.6, 23.0]",67.6,1.0,2.7,1,1.0,1.0,35.15,True,32.45,32.45
3,0,0,3,"[49.0, 61.6]","[7.7, 45.8, 8.1, 18.0, 15.6, 23.0]",61.6,1.0,7.7,0,1.0,1.0,34.65,True,26.95,26.95
4,0,0,4,"[32.0, nan]","[32.5, 45.8, 8.1, 18.0, 15.6, 23.0]",32.0,0.0,8.1,2,1.0,1.0,20.05,True,66.95,11.95
5,0,0,5,"[49.0, nan]","[32.5, 45.8, 14.1, 18.0, 15.6, 23.0]",49.0,0.0,14.1,2,1.0,1.0,31.55,True,7.25,17.45
6,0,0,6,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00
7,0,0,7,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00
8,0,0,8,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00



500


...


5000


,bprofit
currentBidIdx,
0.0,140.95
1.0,112.30


,sprofit
currentAskIdx,
0,12.15
1,29.45
2,37.90
3,50.00
4,0.00
5,32.85


,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
45000,0,5000,0,"[86.0, 100.0]","[7.7, 2.7, 8.1, 0.0, 15.6, 1.9]",100.0,1.0,0.0,3,True,True,50.00,True,50.00,50.00
45001,0,5000,1,"[32.0, 67.6]","[7.7, 2.7, 8.1, 18.0, 15.6, 1.9]",67.6,1.0,1.9,5,True,True,34.75,True,32.85,32.85
45002,0,5000,2,"[32.0, 61.6]","[7.7, 2.7, 8.1, 18.0, 15.6, 23.0]",61.6,1.0,2.7,1,True,True,32.15,True,29.45,29.45
45003,0,5000,3,"[32.0, nan]","[7.7, 45.8, 8.1, 18.0, 15.6, 23.0]",32.0,0.0,7.7,0,True,True,19.85,True,75.25,12.15
45004,0,5000,4,"[49.0, nan]","[32.5, 45.8, 8.1, 18.0, 15.6, 23.0]",49.0,0.0,8.1,2,True,True,28.55,True,58.45,20.45
45005,0,5000,5,"[49.0, nan]","[32.5, 45.8, 14.1, 18.0, 15.6, 23.0]",49.0,0.0,14.1,2,True,True,31.55,True,7.25,17.45
45006,0,5000,6,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00
45007,0,5000,7,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00
45008,0,5000,8,"[nan, nan]","[32.5, 45.8, 56.9, 18.0, 15.6, 23.0]",NaN,NaN,15.6,4,NaN,NaN,NaN,False,0.00,0.00


KeyboardInterrupt: 

In [47]:
log.getPeriod(0,500)

,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
4500,0,500,0,"[30.0, 100.0]","[3.5, 0.0, 17.0, 35.0, 14.3, 10.5]",100.0,1.0,0.0,1,True,True,50.00,True,50.00,50.00
4501,0,500,1,"[22.0, 60.6]","[3.5, 26.2, 17.0, 35.0, 14.3, 10.5]",60.6,1.0,3.5,0,True,True,32.05,True,28.55,28.55
4502,0,500,2,"[22.0, 17.7]","[40.5, 26.2, 17.0, 35.0, 14.3, 10.5]",22.0,0.0,10.5,5,True,True,16.25,True,56.45,5.75
4503,0,500,3,"[30.0, 17.7]","[40.5, 26.2, 17.0, 35.0, 14.3, 43.6]",30.0,0.0,14.3,4,True,True,22.15,True,45.55,7.85
4504,0,500,4,"[30.0, 17.7]","[40.5, 26.2, 17.0, 35.0, 18.7, 43.6]",30.0,0.0,17.0,2,True,True,23.50,True,11.10,6.50
4505,0,500,5,"[nan, 17.7]","[40.5, 26.2, 39.1, 35.0, 18.7, 43.6]",17.7,1.0,18.7,4,False,False,NaN,False,0.00,0.00
4506,0,500,6,"[nan, 17.7]","[40.5, 26.2, 39.1, 35.0, 18.7, 43.6]",17.7,1.0,18.7,4,False,False,NaN,False,0.00,0.00
4507,0,500,7,"[nan, 17.7]","[40.5, 26.2, 39.1, 35.0, 18.7, 43.6]",17.7,1.0,18.7,4,False,False,NaN,False,0.00,0.00
4508,0,500,8,"[nan, 17.7]","[40.5, 26.2, 39.1, 35.0, 18.7, 43.6]",17.7,1.0,18.7,4,False,False,NaN,False,0.00,0.00


In [48]:
display(log.getPeriod(rnd, period).tail(200)[['bprofit', 'currentBidIdx']].groupby('currentBidIdx').sum())


,bprofit
currentBidIdx,
0.0,113.10
1.0,78.55


In [49]:


class Reinforcer(Trader):
    def __init__(self, gameData, disclosure, index, buyer, type):
        super().__init__(gameData, disclosure, index, buyer, type)
        self.learningRate = 0.0002
        self.gamma = 0.99
        self.numActions = 100
        self.depth = 1
        self.numStates = 8 + 6 * self.depth
        agentState = [0, -1, -1, -1, -1, -1, -1, -1]
        history = [-1, -1, -1, -1, -1, -1] * self.depth
        self.state = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        self.policy = Policy(self.numStates, self.numActions)

    def observe(self):
        agentState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        history = self.df.iloc[-self.depth:][['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price']].values.reshape(-1,).tolist()
        if len(history) != (6 * self.depth):
            history = [-1] * (6 * self.depth)
        self.state = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        try: 
            self.policy.data.append((self.stepProfits,self.prob[self.action]))
        except:
            print('skipped')
            pass
        
    def bid(self):
        self.stepBid = np.nan
        if self.stepTokenValue > 0:
            self.prob = self.policy(torch.from_numpy(self.state).float())
            self.action = Categorical(self.prob).sample()
            frac = self.action.item()/self.numActions
            self.stepBid = np.round(frac * 100,1)
        return self.stepBid
        
    def ask(self):
        self.stepAsk = np.nan
        if self.stepTokenValue > 0:
            self.prob = self.policy(torch.from_numpy(self.state).float())
            self.action = Categorical(self.prob).sample()    
            frac = self.action.item()/self.numActions
            self.stepAsk = np.round(frac * 100,1)
        return self.stepAs

In [12]:
def generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure):
    buyers, sellers = [], []
    for idx,i in enumerate(buyerStrategies):
        if i == 'TruthTeller':
            buyers.append(TruthTeller(gameData, disclosure, index=idx, buyer=0, type=0)) 
        if i == 'ZeroIntelligence':
            buyers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=0, type=1)) 
        if i == 'REINFORCE':
            buyers.append(REINFORCE(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'PPO':
            buyers.append(PPO(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'SAC':
            buyers.append(SAC(gameData, disclosure, index=idx, buyer=0, type=2)) 
        if i == 'DQN':
            buyers.append(DQN(gameData, disclosure, index=idx, buyer=0, type=2)) 

    for idx,i in enumerate(sellerStrategies):
        if i == 'TruthTeller':
            sellers.append(TruthTeller(gameData, disclosure, index=idx, buyer=1, type=0)) 
        if i == 'ZeroIntelligence':
            sellers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=1, type=1)) 
        if i == 'REINFORCE':
            sellers.append(REINFORCE(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'PPO':
            sellers.append(PPO(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'SAC':
            sellers.append(SAC(gameData, disclosure, index=idx, buyer=1, type=2)) 
        if i == 'DQN':
            sellers.append(DQN(gameData, disclosure, index=idx, buyer=1, type=2)) 
    return buyers, sellers

In [ ]:
def algorithmSelector(algorithm, args):
    if algorithm == 'REINFORCE':
        return REINFORCE(*args)
    if algorithm == 'DQN':
        return REINFORCE(*args)
    if algorithm == 'PPO':
        return REINFORCE(*args)
    if algorithm == 'SAC':
        return REINFORCE(*args)
    if algorithm == 'DDPG':
        return REINFORCE(*args)

In [ ]:
class Reinforcer(Trader):
    def __init__(self, gameData, disclosure, index, buyer, type,
                 disclosure = ['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price'],
                 algorithm='REINFORCE', depth = 1, numActions = 100, learningRate=0.0002, gamma=0.99):
        super().__init__(gameData, disclosure, index, buyer, type)
        self.learningRate = learningRate
        self.gamma = gamma
        self.numActions = 100
        self.depth = depth
        self.disclosure = disclosure
        self.numStates = 8 + len(self.disclosure) * self.depth
        self.state = np.nan_to_num(np.array[-1] * self.numStates, dtype = np.float32), nan=-9)
        self.algorithm = algorithmSelector(algorithm, [self.numStates, self.numActions, self.learningRate, self.gamma, self.depth, self.disclosure])

    def observe(self):
        internalState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        activityLog = self.df.iloc[-self.depth:][[self.disclosure]].values.reshape(-1,).tolist()
        if len(activityLog) != (len(self.disclosure) * self.depth):
            activityLog = [-1] * (len(self.disclosure) * self.depth)
        self.state = np.nan_to_num(np.array(internalState + activityLog, dtype = np.float32), nan=-9)
        self.algorithm.data.append(data)
        
    def bid(self):
        self.stepBid = np.nan
        if self.stepTokenValue > 0:
            self.stepBid = self.algorithm.action(self.state)
        return self.stepBid
        
    def ask(self):
        self.stepAsk = np.nan
        if self.stepTokenValue > 0:
            self.stepAsk = self.algorithm.action()
        return self.stepAsK

In [2]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    
    
    for n_epi in range(10000):
        s, _ = env.reset()
        done = False
        
        while not done: # CartPole-v1 forced to terminates at 500 step.
            prob = pi(torch.from_numpy(s).float())
            m = Categorical(prob)
            a = m.sample()
            s_prime, r, done, truncated, info = env.step(a.item())
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r
            
        pi.train_net()
        
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()
    
if __name__ == '__main__':
    main()

# of episode :20, avg score : 21.25
# of episode :40, avg score : 20.2
# of episode :60, avg score : 25.45
# of episode :80, avg score : 22.9
# of episode :100, avg score : 24.9
# of episode :120, avg score : 30.65
# of episode :140, avg score : 32.85
# of episode :160, avg score : 28.45
# of episode :180, avg score : 29.45
# of episode :200, avg score : 37.0
# of episode :220, avg score : 29.95
# of episode :240, avg score : 32.5


KeyboardInterrupt: 